In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import json
from model import BigramLanguageModel
from train_model import EncodingDecoding, save_encoder

In [2]:
torch.manual_seed(1337)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Hyperparameters based on device
if device == 'cuda':
    batch_size = 32
    max_iteration = 5000
    block_size = 256
    learning_rate = 3e-4
    eval_interval = 500
    n_embed = 384
    dropout = 0.2
    n_head = 6
    n_layer = 6
else:
    batch_size = 32
    max_iteration = 3000
    block_size = 128
    eval_iters = 100
    learning_rate = 2e-4
    eval_interval = 300
    n_embed = 128
    dropout = 0.1
    n_head = 4
    n_layer = 4

Using device: cpu


In [3]:
file_path = "reddit_chat.jsonl"

Read the file

In [ ]:
conversations = []
with open(file_path, 'r') as f:
    for line in f:
        data = json.loads(line)
        messages = data['message']
        formatted = ""
        for msg in messages:
            if msg['role'] == 'user':
                formatted += f"Human: {msg['content']}\n"
            else:
                formatted += f"Assistant: {msg['content']}\n"
        conversations.append(formatted.strip())

print(f"Loaded {len(conversations)} conversations")

Split Train and Test

In [5]:
all_text = "\n".join(conversations)
data_encoder = EncodingDecoding(all_text)
vocab_size = len(data_encoder.stoi)
print(f"Vocabulary size: {vocab_size}")

Train -->  Dataset({
    features: ['text'],
    num_rows: 47979
})
____________________
Test -->  Dataset({
    features: ['text'],
    num_rows: 5331
})


In [ ]:
save_encoder(data_encoder, "encoder_vocab.pt")

Tokenizer - to encode them

In [6]:
train_text = "\n\n".join(conversations[:45000])  # Use 45k for training
val_text = "\n\n".join(conversations[45000:])    # Rest for validation

train_data = torch.tensor(data_encoder.encode(train_text), dtype=torch.long)
val_data = torch.tensor(data_encoder.encode(val_text), dtype=torch.long)

print(f"Train data: {len(train_data):,} tokens")
print(f"Val data: {len(val_data):,} tokens")

Train sequences - Min: 47, Max: 478, Avg: 159.5
Block size: 128
Sequences longer than block_size: 33786


Get Batch

In [ ]:
# Data loading function - get random chunks
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x.to(device), y.to(device)

Fine-tune the model

In [ ]:
@torch.no_grad()
def estimate_loss(eval_iters=200):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, targets=Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [9]:
print(f"\nInitializing model...")
model = BigramLanguageModel(
    vocab_size=vocab_size,
    n_embed=n_embed,
    block_size=block_size,
    n_head=n_head,
    n_layer=n_layer,
    dropout=dropout,
    device=device
)
model.to(device)
print(f"Model has {sum(p.numel() for p in model.parameters()):,} parameters")


Vocabulary size: 973
Model initialized with 1058253 parameters


In [10]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop
print(f"\nStarting training for {max_iteration} iterations...")
print(f"Block size: {block_size}, Batch size: {batch_size}, LR: {learning_rate}")
print("-" * 60)

for iteration in range(max_iteration):
    # Evaluate periodically
    if iteration % eval_interval == 0 or iteration == max_iteration - 1:
        losses = estimate_loss()
        print(f"Step {iteration:5d}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    # Get batch and train
    xb, yb = get_batch('train')
    logits, loss = model(xb, targets=yb)
    
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print("\nTraining complete!")

KeyboardInterrupt: 

In [ ]:
model_path = "gpt_finetuned_reddit.pt"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

print("\n" + "="*60)
print("Quick generation test:")
print("="*60)

model.eval()
context = torch.tensor([data_encoder.encode("Human: Hello!\nAssistant: ")], dtype=torch.long, device=device)

with torch.no_grad():
    generated = model.generate(context, max_new_tokens=100)

result = data_encoder.decode(generated[0].tolist())
print(result)